# Ex. 3.2 HBASE
We use the *HappyBase* lib: https://happybase.readthedocs.io/en/latest/installation.html

In [1]:
!pip install -q condacolab

In [2]:
import condacolab 
condacolab.install()
!conda install -y pyhive
!conda install -c conda-forge -y happybase

✨🍰✨ Everything looks OK!
Solving environment: | / - \ | / - \ | / - \ | / - done

# All requested packages already installed.

Solving environment: | / - \ | / - \ | / - \ | / - done

# All requested packages already installed.



In [3]:
# imports
import happybase

In [4]:
# try to connect
server="ec2-34-254-248-152.eu-west-1.compute.amazonaws.com" #need to change this for every lab-session
connection = happybase.Connection(server)

In [5]:
families = {
    'cf1': dict(max_versions=10),
    'cf2': dict(max_versions=1, block_cache_enabled=False),
    'cf3': dict(),  # use defaults
}

In [6]:
#connection.create_table('lab3', families)  # table allredy exists

In [7]:
print(connection.tables())

[b'1234_user_id_employees', b'49a5b1a_employees', b'dh_user_id_employees', b'employee_Group2', b'employee_Group_2', b'employee_group_five', b'employees_smunaret', b'group1_employee', b'group3_user_id_employees', b'lab3', b'mspitzna_employees', b'tm_user_id_employees', b'user_id_employees']


In [8]:
#connect to a table
table = connection.table('lab3')

In [9]:
#see column families
table.families()

{b'cf1': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 10,
  'name': b'cf1:',
  'time_to_live': 2147483647},
 b'cf2': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 1,
  'name': b'cf2:',
  'time_to_live': 2147483647},
 b'cf3': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 3,
  'name': b'cf3:',
  'time_to_live': 2147483647}}

In [10]:
#insert something
table.put('row 6', {'cf1:col7': '17.33', 'cf1:col8': '344'})


In [11]:
#read table
res=table.scan()

In [12]:
for key,row in res:
    print(key,row)

b'10001' {b'cf1:col1': b"'1953-09-02'", b'cf1:col2': b"'Georgi'", b'cf1:col3': b"'Facello'", b'cf1:col4': b"'M'", b'cf2:col5': b"'1986-06-26'"}
b'10002' {b'cf1:col1': b"'1964-06-02'", b'cf1:col2': b"'Bezalel'", b'cf1:col3': b"'Simmel'", b'cf1:col4': b"'F'", b'cf2:col5': b"'1985-11-21'"}
b'10003' {b'cf1:col1': b"'1959-12-03'", b'cf1:col2': b"'Parto'", b'cf1:col3': b"'Bamford'", b'cf1:col4': b"'M'", b'cf2:col5': b"'1986-08-28'"}
b'10004' {b'cf1:col1': b"'1954-05-01'", b'cf1:col2': b"'Chirstian'", b'cf1:col3': b"'Koblick'", b'cf1:col4': b"'M'", b'cf2:col5': b"'1986-12-01'"}
b'10005' {b'cf1:col1': b"'1955-01-21'", b'cf1:col2': b"'Kyoichi'", b'cf1:col3': b"'Maliniak'", b'cf1:col4': b"'M'", b'cf2:col5': b"'1989-09-12'"}
b'10006' {b'cf1:col1': b"'1953-04-20'", b'cf1:col2': b"'Anneke'", b'cf1:col3': b"'Preusig'", b'cf1:col4': b"'F'", b'cf2:col5': b"'1989-06-02'"}
b'10007' {b'cf1:col1': b"'1957-05-23'", b'cf1:col2': b"'Tzvetan'", b'cf1:col3': b"'Zielinski'", b'cf1:col4': b"'F'", b'cf2:col5': b"

# Ex 1
Employee data in HBASE.
* create a new table to hold the employee data -> use a distinct name, like ``user_id_employees``
* How would you create the column families?
* read data from ``employees.csv`` and put it into the table ->  use  ``import csv`` to read the data -> [API docs](https://docs.python.org/3/library/csv.html)
* use batch-processing to enter more than one entry per transaction
* alter some entries of the table with ``timestamps``
* perform some sample scans for the latest entries

In [13]:
connection.create_table(
    'user_id_employees',
    {'cf1': dict(max_versions=10),
     'cf2': dict(max_versions=1, block_cache_enabled=False),
     'cf3': dict(),  # use defaults
    }
)

AlreadyExists: ignored

In [14]:
table = connection.table('user_id_employees')

In [15]:
print(connection.tables())

[b'1234_user_id_employees', b'49a5b1a_employees', b'dh_user_id_employees', b'employee_Group2', b'employee_Group_2', b'employee_group_five', b'employees_smunaret', b'group1_employee', b'group3_user_id_employees', b'lab3', b'mspitzna_employees', b'tm_user_id_employees', b'user_id_employees']


In [16]:
import csv
csvfile = open('/content/employees.csv', "r")
csvreader = csv.reader(csvfile)

In [17]:
batch = table.batch(batch_size = 100)

In [18]:
def insert_row(batch, row):
    """ Insert a row into HBase.
    Write the row to the batch. When the batch size is reached, rows will be
    sent to the database.
    Rows have the following schema:
        [ id, keyword, subcategory, type, township, city, zip, council_district,
          opened, closed, status, origin, location ]
    """
    batch.put(row[0], { "data:id": row[1], "data:birth": row[2], "data:name": row[3],
        "data:vorname": row[4], "data:gender": row[5]})

In [21]:
row_count = 0

try:
    # Loop through the rows. The first row contains column headers, so skip that
    # row. Insert all remaining rows into the database.
    for row in csvreader:
        row_count += 1
        if row_count == 1:
            pass
        else:
            insert_row(batch, row)

    # If there are any leftover rows in the batch, send them now.
    batch.send()
finally:
    # No matter what happens, close the file handle.
    csvfile.close()


IOError: ignored